In [1]:
# (hypothetical) state populations
p = [27744, 25178, 19951, 14610, 9225, 3292]

# number of states
n = len(p)

# total country population
p_total = sum( p[i] for i in range(n) )

# total number of seats to distribute
k = 36

# state quotas
q = [ k * p[i] / p_total for i in range(n) ]

print("Populations:",p)
print("Quotas:",q)

Populations: [27744, 25178, 19951, 14610, 9225, 3292]
Quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# create model object
m = gp.Model()

# create integer vars x, with x[i] being the number of seats for state i
x = m.addVars( n, vtype=GRB.INTEGER )

# distribute k seats
m.addConstr( gp.quicksum( x[i] for i in range(n) ) == k )

# each state gets at least one seat
for i in range(n):
    x[i].LB = 1

m.update()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-06-27
Using license file C:\Users\buchanan\gurobi.lic


In [4]:
# Variable y introduced to linearize the max term y = (max_i x_i/p_i)
y = m.addVar()
m.addConstrs( x[i] <= p[i] * y for i in range(n) )

# Add objective function for Jefferson's method: min_x max_i { x_i / p_i }
m.setObjective( y, GRB.MINIMIZE )

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 7 columns and 18 nonzeros
Model fingerprint: 0x1bffcd12
Variable types: 1 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 4e+01]
Presolve time: 0.00s
Presolved: 7 rows, 7 columns, 18 nonzeros
Variable types: 1 continuous, 6 integer (0 binary)
Found heuristic solution: objective 0.0094168

Root relaxation: objective 3.600000e-04, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00036    0    6    0.00942    0.00036  96.2%     -    0s
H    0     0                       0.0003972    0.00036  9.36%     -    0s
*    0     0               0       0.0003

In [5]:
sol = [ x[i].x for i in range(n) ]
print("Optimal solution :", sol)
print("Compare to quotas:", q)

Optimal solution : [11.0, 9.0, 7.0, 5.0, 3.0, 1.0]
Compare to quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]
